# CareerForge AI Agent

In [ ]:
# Install All dependencies for agent
import os
import sys
from google.genai import types
from google.adk.tools import Tool

# --- TOOL 1: Google Search (Built-in Wrapper) ---
# Assuming your ADK kit provides a wrapper, or we use the GenAI tool
from google.genai.types import Tool as GenAiTool
from google.genai.types import GoogleSearchRetrieval
from google.adk.agents import Agent  # Using the class you requested
from google.adk.models.google_llm import Gemini
from IPython.display import display, Markdown

In [5]:
# --- 1. AUTHENTICATION ---
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    # Fallback for local testing
    os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE" 
    print(f"⚠️ Using fallback/local key setup.")

print("✅ API key loaded loaded")

✅ Gemini API key setup complete via Kaggle Secrets.
❌ Critical Error: 'google.adk' library not found. Ensure you are in the Course Environment.


In [ ]:
# --- 2. RETRY CONFIGURATION (ROBUSTNESS) ---
# This ensures the agent doesn't crash if the API is busy
retry_config = types.HttpRetryOptions(
    attempts=5,                  # Try 5 times before failing
    exp_base=7,                  # Exponential backoff multiplier
    initial_delay=1,             # Start waiting 1 second
    http_status_codes=[429, 500, 503, 504], # Catch rate limits and server errors
)

print("✅ Retry configuration loaded.")

In [ ]:
# Define the search tool capability for Gemini
google_search_tool = GenAiTool(
    google_search_retrieval=GoogleSearchRetrieval()
)

# --- TOOL 2: Custom ATS Calculator (Python Logic) ---
def calculate_ats_match(resume_text: str, keywords: list[str]):
    """
    Calculates the match percentage of keywords in a resume.
    """
    if not keywords: return {"score": 0, "missing": []}
    
    resume_lower = resume_text.lower()
    found = [word for word in keywords if word.lower() in resume_lower]
    score = (len(found) / len(keywords)) * 100
    missing = [word for word in keywords if word not in found]
    
    return {
        "score": round(score, 2), 
        "missing_keywords": missing,
        "status": "PASS" if score > 70 else "FAIL"
    }

print("✅ Tools defined: Google Search & ATS Calculator.")

In [ ]:
# --- AGENT 1: COMPANY RESEARCHER (Uses Google Search) ---
# Task: Find out about the company culture so the interview is realistic.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-1.5-flash", # Use 1.5-flash for speed/cost
        retry_options=retry_config # Applying the retry logic
    ),
    instruction="""
    You are a Corporate Researcher. 
    1. Extract the Company Name from the Job Description provided in the prompt.
    2. Use the `google_search` tool to find the company's "Values", "Mission", or recent news.
    3. Summarize 3 key facts about the company culture.
    """,
    tools=[google_search_tool], # Uses Search
    output_key="company_research" # Saves result to session state
)

# --- AGENT 2: ATS AUDITOR (Uses Custom Code) ---
# Task: Check if the resume matches the keywords.
audit_agent = Agent(
    name="AuditAgent",
    model=Gemini(
        model="gemini-1.5-flash", 
        retry_options=retry_config
    ),
    instruction="""
    You are a Technical Recruiter.
    1. Identify top 5 technical keywords from the Job Description.
    2. CALL the `calculate_ats_match` tool using the Resume and those keywords.
    3. Output the Score and a strict advice on what to fix.
    """,
    tools=[calculate_ats_match], # Uses Custom Python Tool
    output_key="ats_report"
)

# --- AGENT 3: THE INTERVIEWER (Context Aware) ---
# Task: Ask questions based on the Research and the Resume gaps.
interview_agent = Agent(
    name="InterviewAgent",
    model=Gemini(
        model="gemini-1.5-pro", # Uses Pro for better reasoning
        retry_options=retry_config
    ),
    instruction="""
    You are a Hiring Manager.
    1. Read the 'company_research' to understand our culture.
    2. Read the 'ats_report' to see the candidate's weak points.
    3. Start a mock interview. Ask ONE hard question that tests a missing skill or cultural fit.
    """,
    # No tools needed, just pure reasoning + context
    output_key="interview_question"
)

print("✅ Agents created: Researcher, Auditor, Interviewer.")

In [ ]:
# Mock Data for Testing
job_description = """
Software Engineer at Google. 
Must know Python, Kubernetes, and Golang. 
Experience with distributed systems is required.
"""

user_resume = """
Jane Doe. Python Developer.
Experience with Django and Flask.
I love coding.
"""

# --- ORCHESTRATION ---
print("🚀 CareerForge AI Starting...\n")

# Shared State (Memory)
session_state = {
    "job_description": job_description,
    "resume": user_resume
}

# 1. Run Research Agent
print("🌍 Step 1: Researching Company...")
research_prompt = f"Analyze this JD: {job_description}"
# Assuming .run() returns a result object or string
research_result = research_agent.run(research_prompt) 
session_state["company_research"] = research_result.text # Store manually if not auto-stored
print(f"   > Findings: {research_result.text[:100]}...")

# 2. Run Audit Agent
print("\n⚙️ Step 2: Auditing Resume...")
audit_prompt = f"JD: {job_description} \n Resume: {user_resume}"
audit_result = audit_agent.run(audit_prompt)
session_state["ats_report"] = audit_result.text
print(f"   > Audit Score: {audit_result.text}")

# 3. Run Interview Agent (Using Context)
print("\n🎤 Step 3: Starting Interview...")
# We inject the outputs of previous agents into the prompt context
interview_context_prompt = f"""
CONTEXT:
Company Info: {session_state['company_research']}
ATS Report: {session_state['ats_report']}
Candidate Resume: {user_resume}

Start the interview now.
"""

interview_result = interview_agent.run(interview_context_prompt)

display(Markdown(f"### 🤖 Interviewer Question:\n{interview_result.text}"))